In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import glob
from tqdm import tqdm
import math
from sklearn.neighbors import KernelDensity
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import folium

In [ ]:
def latlonTable2GDF(table: pd.DataFrame, colName_lat: str, colName_lon: str):
    return gpd.GeoDataFrame(table, geometry=gpd.points_from_xy(table[colName_lon], table[colName_lat])).set_crs(6668)
def haversine_distance(coord1, coord2):
    """
    緯度・経度の2点間の直線距離（ハバーサインの公式）
    :param coord1: (lat1, lon1) - 点1の緯度・経度
    :param coord2: (lat2, lon2) - 点2の緯度・経度
    :return: 2点間の距離（単位：メートル）
    """
    R = 6371.0 * 1000  # 地球の半径（メートル）

    lat1, lon1 = map(math.radians, coord1)
    lat2, lon2 = map(math.radians, coord2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

In [ ]:
perf_no = "14"
pref_name = "神奈川県"
# 対象地区（横浜市）
target_jcodes = [
    "14101",
    "14102",
    "14103",
    "14104",
    "14105",
    "14106",
    "14107",
    "14108",
    "14109",
    "14110",
    "14111",
    "14112",
    "14113",
    "14114",
    "14115",
    "14116",
    "14117",
    "14118"
]
path = f"Y:\\GIS\\ゼンリンデータ\\建物ポイント\\*\\*"

In [ ]:
target_paths = []
for p in tqdm(glob.glob(path)):
    jcode = p.split("\\")[-1][3:8]
    if jcode in target_jcodes:
        target_paths.append(p)

In [ ]:
dfs = []
for p in tqdm(target_paths):
    dfs.append(pd.read_csv(p,usecols=["市区町村名","大字名","字丁目名","街区","地番・戸番","建物名","階数","建物分類","面積","緯度","経度"],encoding="shift_jis"))
dfs = pd.concat(dfs)

In [ ]:
dfs["address"] = (
        dfs["市区町村名"].fillna('') + "_" +
        dfs["大字名"].fillna('') + "_" +
        dfs["字丁目名"].fillna('') + "_" +
        dfs["街区"].fillna('') + "_" +
        dfs["地番・戸番"].fillna('') + "_" +
        dfs["建物名"].fillna('') + "_" +
        dfs["階数"].fillna('').astype(str)
)
dfs = dfs.filter(items=["建物分類","面積","緯度","経度","address"]).set_axis(["genre","area","latitude","longitude","address"],axis=1)
house = [1001,1002,1003,1004,1005,1006,1007,1008,9999]
non_house = dfs.query("genre not in @house")

In [ ]:
JCODE = "14100"
genre_master = pd.read_csv("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\zenrin_genre_master.csv",usecols=["建物分類","産業大分類","モデル用定義"]).set_axis(["genre","industry_type","behavior_type"],axis=1)
pd.merge(non_house,genre_master,on="genre",how="left").filter(items=["genre","area","latitude","longitude","address","industry_type","behavior_type"]).to_csv(f"C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\zenrin_{JCODE}.csv",index=False)

In [ ]:
# non_house.to_csv("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\zenrin_14100.csv")
non_house = pd.read_csv("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\zenrin_14100.csv")

対象範囲にclip

In [ ]:
mesh_poly = gpd.read_file("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\mesh\\mesh_geom.shp")

In [ ]:
sjoined = latlonTable2GDF(non_house,"latitude","longitude").sjoin(mesh_poly.to_crs(6668),how="left",predicate="intersects").filter(items=["genre","area","latitude","longitude","address","mesh_code","industry_type","behavior_type"])

In [ ]:
# 対象地域内
sjoined.query("mesh_code == mesh_code").to_csv("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\poi\\zenrin_building.csv",index=False)

In [ ]:
sjoined

In [ ]:
# 対象地域外
out_area = sjoined.query("mesh_code != mesh_code").copy()
out_area["eval"] = out_area.apply(
    lambda row: row["area"]/haversine_distance((centLat,centLon),(row["latitude"],row["longitude"])),axis=1)

In [ ]:
cs = out_area.query("genre == 2010").copy()

In [ ]:
# データの読み込み
latlon = np.vstack([cs.latitude, cs.longitude]).T
values = cs["eval"]  # 評価値を格納しているカラム

# カーネル密度推定の実行
kde = KernelDensity(bandwidth=0.01, kernel='gaussian')
kde.fit(latlon, sample_weight=values)  # 評価値をサンプルウェイトとして使用
x_d = np.linspace(latlon[:, 0].min(), latlon[:, 0].max(), 100)
y_d = np.linspace(latlon[:, 1].min(), latlon[:, 1].max(), 100)
xv, yv = np.meshgrid(x_d, y_d)
xy_sample = np.vstack([xv.ravel(), yv.ravel()]).T
density = np.exp(kde.score_samples(xy_sample)).reshape(100, 100)

# 密度分布の可視化
plt.imshow(density, extent=(x_d.min(), x_d.max(), y_d.min(), y_d.max()), origin='lower')
plt.colorbar(label='Density')
plt.show()

# ローカルピークの検出
peaks, _ = find_peaks(density.ravel(), prominence=0.8)  # prominenceは調整可能
peak_positions = np.unravel_index(peaks, density.shape)

# ローカルピークに対応する緯度経度
peak_latlon = np.vstack([x_d[peak_positions[1]], y_d[peak_positions[0]]]).T
print(peak_latlon)

In [ ]:
# foliumの地図作成（最初のピーク地点を中心に設定）
m = folium.Map(location=[peak_latlon[0][0], peak_latlon[0][1]], zoom_start=12)

# ローカルピークを地図に追加
for lat, lon in peak_latlon:
    folium.Marker([lat, lon], popup=f'Peak: ({lat}, {lon})').add_to(m)

# 地図を表示
m